#### BrisT1D Blood Glucose Prediction Competition
Using historical blood glucose readings, insulin dosage, carbohydrate intake, and smartwatch activity data to predict future blood glucose

Kaggle - https://www.kaggle.com/competitions/brist1d.

In [1]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
# Load datasets
train_data = TabularDataset('train.csv')
test_data = TabularDataset('test.csv')

In [3]:
train_data.head()

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


In [4]:
train_data.shape

(177024, 508)

In [5]:
test_data.shape

(3644, 507)

In [6]:
# Function to optimize memory usage by downcasting
def optimize_memory(df):
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['int']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

In [7]:
# train_data = optimize_memory(train_data)
# test_data = optimize_memory(test_data)

In [8]:
# Define the target column for prediction
target = 'bg+1:00'

In [9]:
# Initialize the AutoGluon TabularPredictor for regression
predictor = TabularPredictor(
    label=target,                   # Target column for blood glucose level prediction
    eval_metric='rmse',             # Evaluation metric as required by the competition
    problem_type='regression'       # Set to regression for continuous value prediction
).fit(
    train_data,
    presets='best_quality',         # Best quality preset for more accurate predictions
    time_limit=3600 * 11,           # Training time limit (11 hours, adjust as needed)
    verbosity=2,                    # Detailed logs for tracking
    excluded_model_types=['KNN'],   # Exclude KNN for efficiency on large datasets
    ag_args_fit={'num_cpus': 10, 'memory_ratio': 0.8}  # Optimized CPU, GPU, and memory usage
)

# Summarize results after training
results = predictor.fit_summary()

No path specified. Models will be saved in: "AutogluonModels\ag-20241107_164552"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       21.51 GB / 31.82 GB (67.6%)
Disk Space Avail:   644.78 GB / 953.00 GB (67.7%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdo

(_ray_fit pid=26080) [1000]	valid_set's rmse: 1.70433
(_ray_fit pid=26080) [2000]	valid_set's rmse: 1.62611 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=26080) [3000]	valid_set's rmse: 1.57209 [repeated 2x across cluster]
(_ray_fit pid=26080) [4000]	valid_set's rmse: 1.53086 [repeated 2x across cluster]
(_ray_fit pid=26080) [5000]	valid_set's rmse: 1.49829 [repeated 2x across cluster]
(_ray_fit pid=26184) [5000]	valid_set's rmse: 1.5009
(_ray_fit pid=26080) [6000]	valid_set's rmse: 1.46952
(_ray_fit pid=26184) [7000]	valid_set's rmse: 1.45227 [repeated 2x across cluster]
(_ray_fit pid=26184) [8000]	valid_set's rmse: 1.43268 [repeated 2x across cluster]
(_ray_fit pid=26184) [9000]	valid_set's rmse: 1.41625 [repeated 2x across cluster]
(_ray_fit pid=26184) [10000]	valid_set's rmse: 1.40117 [repeate

(_dystack pid=6300) 	-1.4121	 = Validation score   (-root_mean_squared_error)
(_dystack pid=6300) 	2162.05s	 = Training   runtime
(_dystack pid=6300) 	26.96s	 = Validation runtime
(_dystack pid=6300) Fitting model: LightGBM_BAG_L1 ... Training model for up to 4413.16s of the 7707.82s of remaining time.
(_dystack pid=6300) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 16.73% memory usage per fold, 66.91%/80.00% total).
(_dystack pid=6300) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=10, gpus=0, memory=16.73%)


(_ray_fit pid=19104) [1000]	valid_set's rmse: 1.70706 [repeated 2x across cluster]
(_ray_fit pid=19104) [2000]	valid_set's rmse: 1.62497 [repeated 2x across cluster]
(_ray_fit pid=19104) [3000]	valid_set's rmse: 1.57349 [repeated 2x across cluster]
(_ray_fit pid=19104) [4000]	valid_set's rmse: 1.53253 [repeated 2x across cluster]
(_ray_fit pid=19104) [5000]	valid_set's rmse: 1.49672 [repeated 2x across cluster]
(_ray_fit pid=19104) [6000]	valid_set's rmse: 1.47125 [repeated 2x across cluster]
(_ray_fit pid=19104) [7000]	valid_set's rmse: 1.45009 [repeated 2x across cluster]
(_ray_fit pid=19104) [8000]	valid_set's rmse: 1.43142 [repeated 2x across cluster]
(_ray_fit pid=17116) [9000]	valid_set's rmse: 1.40544 [repeated 2x across cluster]
(_ray_fit pid=19104) [10000]	valid_set's rmse: 1.4039 [repeated 2x across cluster]
(_ray_fit pid=9768) [1000]	valid_set's rmse: 1.73134 [repeated 2x across cluster]
(_ray_fit pid=9768) [2000]	valid_set's rmse: 1.65168 [repeated 2x across cluster]
(_ray_

(_dystack pid=6300) 	-1.4141	 = Validation score   (-root_mean_squared_error)
(_dystack pid=6300) 	1270.29s	 = Training   runtime
(_dystack pid=6300) 	24.38s	 = Validation runtime
(_dystack pid=6300) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 3136.77s of the 6431.43s of remaining time.
(_dystack pid=6300) 	-1.6309	 = Validation score   (-root_mean_squared_error)
(_dystack pid=6300) 	1540.43s	 = Training   runtime
(_dystack pid=6300) 	26.73s	 = Validation runtime
(_dystack pid=6300) Fitting model: CatBoost_BAG_L1 ... Training model for up to 1568.13s of the 4862.79s of remaining time.
(_dystack pid=6300) 	Memory not enough to fit 8 folds in parallel. Will train 2 folds in parallel instead (Estimated 20.02% memory usage per fold, 40.05%/80.00% total).
(_dystack pid=6300) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=10, gpus=0, memory=20.02%)
(_dystack pid=6300) 	-1.8581	 = Validation score   (-root_mea

...

(_dystack pid=6300) 	-1.1013	 = Validation score   (-root_mean_squared_error)
(_dystack pid=6300) 	1158.64s	 = Training   runtime
(_dystack pid=6300) 	22.24s	 = Validation runtime
(_dystack pid=6300) Fitting model: LightGBM_BAG_L2 ... Training model for up to 1535.13s of the 1535.01s of remaining time.
(_dystack pid=6300) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 16.61% memory usage per fold, 66.44%/80.00% total).
(_dystack pid=6300) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=10, gpus=0, memory=16.61%)


(_ray_fit pid=27276) [1000]	valid_set's rmse: 1.17336 [repeated 2x across cluster]
(_ray_fit pid=27276) [2000]	valid_set's rmse: 1.15372 [repeated 2x across cluster]
(_ray_fit pid=17884) [3000]	valid_set's rmse: 1.14391 [repeated 2x across cluster]
(_ray_fit pid=17884) [4000]	valid_set's rmse: 1.13365 [repeated 2x across cluster]
(_ray_fit pid=17884) [5000]	valid_set's rmse: 1.12676 [repeated 2x across cluster]
(_ray_fit pid=17884) [6000]	valid_set's rmse: 1.12159 [repeated 2x across cluster]
(_ray_fit pid=17884) [7000]	valid_set's rmse: 1.11693 [repeated 2x across cluster]
(_ray_fit pid=17884) [8000]	valid_set's rmse: 1.11272 [repeated 2x across cluster]
(_ray_fit pid=17884) [9000]	valid_set's rmse: 1.10914 [repeated 2x across cluster]
(_ray_fit pid=17884) [10000]	valid_set's rmse: 1.10616 [repeated 2x across cluster]
(_ray_fit pid=20720) [1000]	valid_set's rmse: 1.17601 [repeated 2x across cluster]
(_ray_fit pid=20720) [2000]	valid_set's rmse: 1.15777 [repeated 2x across cluster]
(_r

(_dystack pid=6300) 	-1.1044	 = Validation score   (-root_mean_squared_error)
(_dystack pid=6300) 	1193.27s	 = Training   runtime
(_dystack pid=6300) 	19.33s	 = Validation runtime
(_dystack pid=6300) Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 336.55s of the 336.43s of remaining time.
(_dystack pid=6300) 	-1.193	 = Validation score   (-root_mean_squared_error)
(_dystack pid=6300) 	1642.6s	 = Training   runtime
(_dystack pid=6300) 	26.24s	 = Validation runtime
(_dystack pid=6300) Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -1334.58s of remaining time.
(_dystack pid=6300) 	Ensemble Weights: {'LightGBMXT_BAG_L2': 0.538, 'LightGBM_BAG_L2': 0.462}
(_dystack pid=6300) 	-1.0918	 = Validation score   (-root_mean_squared_error)
(_dystack pid=6300) 	0.28s	 = Training   runtime
(_dystack pid=6300) 	0.0s	 = Validation runtime
(_dystack pid=6300) AutoGluon training complete, total runtime = 11230.39s ... Best model: WeightedEnsemble_L3 | Esti

*** Summary of fit() ***
Estimated performance of each model:
                     model  score_val              eval_metric  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L3  -1.048201  root_mean_squared_error     194.085680  21123.480018                0.002998           0.462963            3       True         17
1        LightGBMXT_BAG_L2  -1.057460  root_mean_squared_error     167.005683  19915.854228               26.878571        1154.956234            2       True         10
2          LightGBM_BAG_L2  -1.060814  root_mean_squared_error     167.204112  19968.060821               27.076999        1207.162827            2       True         11
3          CatBoost_BAG_L2  -1.133447  root_mean_squared_error     140.658111  23013.314626                0.530998        4252.416632            2       True         13
4           XGBoost_BAG_L2  -1.145858  root_mean_squared_error     145.317335  18978.404

C:\Users\Hi\anaconda3\envs\gpu_env\lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [10]:
# Prepare test data without the target column for prediction
test_data_no_target = test_data.drop(columns=[target], errors='ignore')

In [11]:
# Make predictions on the test data
predictions = predictor.predict(test_data_no_target)

In [12]:
# Prepare the submission file
submission = pd.DataFrame({'id': test_data['id'], target: predictions})
submission.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv.")

Submission file saved as submission.csv.


In [13]:
results

{'model_types': {'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestMSE_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesMSE_BAG_L1': 'StackerEnsembleModel_XT',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB',
  'RandomForestMSE_BAG_L2': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L2': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesMSE_BAG_L2': 'StackerEnsembleModel_XT',
  'XGBoost_BAG_L2': 'StackerEnsembleModel_XGBoost',
  'LightGBMLarge_BAG_L2': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L3': 'WeightedEnsembleModel'},
 'model_performance': {'LightGBMXT_BAG_L1': -1.3930247603230939,
  'LightGB

In [14]:
# Get the leaderboard to see the performance of the models
leaderboard = predictor.leaderboard(silent=True)

# Print the leaderboard for reference
leaderboard

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-1.048201,root_mean_squared_error,194.085680,21123.480018,0.002998,0.462963,3,True,17
1,LightGBMXT_BAG_L2,-1.057460,root_mean_squared_error,167.005683,19915.854228,26.878571,1154.956234,2,True,10
2,LightGBM_BAG_L2,-1.060814,root_mean_squared_error,167.204112,19968.060821,27.076999,1207.162827,2,True,11
3,CatBoost_BAG_L2,-1.133447,root_mean_squared_error,140.658111,23013.314626,0.530998,4252.416632,2,True,13
4,XGBoost_BAG_L2,-1.145858,root_mean_squared_error,145.317335,18978.404713,5.190223,217.506719,2,True,15
5,RandomForestMSE_BAG_L2,-1.151171,root_mean_squared_error,176.467281,20479.365496,36.340168,1718.467502,2,True,12
6,ExtraTreesMSE_BAG_L2,-1.157772,root_mean_squared_error,176.161943,19512.007148,36.034830,751.109153,2,True,14
7,WeightedEnsemble_L2,-1.375363,root_mean_squared_error,49.732991,2291.881114,0.002000,0.384508,2,True,9
8,LightGBM_BAG_L1,-1.390787,root_mean_squared_error,22.883993,1182.164187,22.883993,1182.164187,1,True,2
9,LightGBMXT_BAG_L1,-1.393025,root_mean_squared_error,26.846997,1109.332419,26.846997,1109.332419,1,True,1


In [15]:
# Get the top 10 models based on the leaderboard
top_10_models = leaderboard['model'].head(10).tolist()
top_10_models

['WeightedEnsemble_L3',
 'LightGBMXT_BAG_L2',
 'LightGBM_BAG_L2',
 'CatBoost_BAG_L2',
 'XGBoost_BAG_L2',
 'RandomForestMSE_BAG_L2',
 'ExtraTreesMSE_BAG_L2',
 'WeightedEnsemble_L2',
 'LightGBM_BAG_L1',
 'LightGBMXT_BAG_L1']

In [16]:
# Iterate through the top 10 models and print their hyperparameters
for model in top_10_models:
    hyperparameters = predictor.info()['model_info'][model]['hyperparameters']
    print(f"Model: {model}")
    print("Hyperparameters:")
    print(hyperparameters)
    print("-" * 50)

Model: WeightedEnsemble_L3
Hyperparameters:
{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}
--------------------------------------------------
Model: LightGBMXT_BAG_L2
Hyperparameters:
{'use_orig_features': True, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}
--------------------------------------------------
Model: LightGBM_BAG_L2
Hyperparameters:
{'use_orig_features': True, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}
--------------------------------------------------
Model: CatBoost_BAG_L2
Hyperparameters:
{'use_orig_features': True, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}
--------------------------------------------------
Model: XGBoost_BAG_L2
Hyperparameters:
{'use_orig_features': True, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}
--------------------------------------------------
Model: Ra

In [17]:
# Get details of the best model
best_model = predictor.get_model_best()
print(f"Best Model: {best_model}")

# Get model hyperparameters and other info
best_model_info = predictor.info()
print(best_model_info)

Best Model: WeightedEnsemble_L3


C:\Users\Hi\AppData\Local\Temp\ipykernel_8752\3617745900.py:2: DeprecationWarning: `get_model_best` has been deprecated and will be removed in version 1.2. Please use `model_best` instead. This will raise an error in the future!
  best_model = predictor.get_model_best()


{'path': 'AutogluonModels\\ag-20241107_164552', 'label': 'bg+1:00', 'random_state': 0, 'version': '1.1.1', 'features': ['p_num', 'time', 'bg-5:55', 'bg-5:50', 'bg-5:45', 'bg-5:40', 'bg-5:35', 'bg-5:30', 'bg-5:25', 'bg-5:20', 'bg-5:15', 'bg-5:10', 'bg-5:05', 'bg-5:00', 'bg-4:55', 'bg-4:50', 'bg-4:45', 'bg-4:40', 'bg-4:35', 'bg-4:30', 'bg-4:25', 'bg-4:20', 'bg-4:15', 'bg-4:10', 'bg-4:05', 'bg-4:00', 'bg-3:55', 'bg-3:50', 'bg-3:45', 'bg-3:40', 'bg-3:35', 'bg-3:30', 'bg-3:25', 'bg-3:20', 'bg-3:15', 'bg-3:10', 'bg-3:05', 'bg-3:00', 'bg-2:55', 'bg-2:50', 'bg-2:45', 'bg-2:40', 'bg-2:35', 'bg-2:30', 'bg-2:25', 'bg-2:20', 'bg-2:15', 'bg-2:10', 'bg-2:05', 'bg-2:00', 'bg-1:55', 'bg-1:50', 'bg-1:45', 'bg-1:40', 'bg-1:35', 'bg-1:30', 'bg-1:25', 'bg-1:20', 'bg-1:15', 'bg-1:10', 'bg-1:05', 'bg-1:00', 'bg-0:55', 'bg-0:50', 'bg-0:45', 'bg-0:40', 'bg-0:35', 'bg-0:30', 'bg-0:25', 'bg-0:20', 'bg-0:15', 'bg-0:10', 'bg-0:05', 'bg-0:00', 'insulin-5:55', 'insulin-5:50', 'insulin-5:45', 'insulin-5:40', 'insuli

In [19]:
# Feature importance
feature_importance = predictor.feature_importance(train_data)
print("Feature Importance:\n", feature_importance)

These features in provided data are not utilized by the predictor and will be ignored: ['id']
Computing feature importance via permutation shuffling for 506 features using 5000 rows with 5 shuffle sets...
	49939.95s	= Expected runtime (9987.99s per shuffle set)
	31245.91s	= Actual runtime (Completed 5 of 5 shuffle sets)


Feature Importance:
                importance    stddev       p_value  n  p99_high   p99_low
bg-0:00          2.022384  0.054928  6.523384e-08  5  2.135481  1.909287
bg-0:05          0.185313  0.014624  4.615269e-06  5  0.215423  0.155202
p_num            0.152783  0.022164  5.169048e-05  5  0.198420  0.107146
bg-0:10          0.006658  0.005769  3.064144e-02  5  0.018537 -0.005221
activity-3:15   -0.000207  0.000064  9.990265e-01  5 -0.000075 -0.000339
...                   ...       ...           ... ..       ...       ...
bg-0:35         -0.035206  0.002542  9.999968e-01  5 -0.029972 -0.040441
bg-0:15         -0.040210  0.004456  9.999822e-01  5 -0.031035 -0.049385
bg-0:20         -0.042080  0.001664  9.999997e-01  5 -0.038654 -0.045506
bg-0:45         -0.044628  0.001173  9.999999e-01  5 -0.042212 -0.047044
bg-0:30         -0.061769  0.002243  9.999998e-01  5 -0.057151 -0.066386

[506 rows x 6 columns]


In [20]:
submission.head()

,id,bg+1:00
0,p01_8459,8.651827
1,p01_8460,4.748473
2,p01_8461,9.101196
3,p01_8462,11.132580
4,p01_8463,6.888596


In [21]:
feature_importance

,importance,stddev,p_value,n,p99_high,p99_low
bg-0:00,2.022384,0.054928,6.523384e-08,5,2.135481,1.909287
bg-0:05,0.185313,0.014624,4.615269e-06,5,0.215423,0.155202
p_num,0.152783,0.022164,5.169048e-05,5,0.198420,0.107146
bg-0:10,0.006658,0.005769,3.064144e-02,5,0.018537,-0.005221
activity-3:15,-0.000207,0.000064,9.990265e-01,5,-0.000075,-0.000339
...,...,...,...,...,...,...
bg-0:35,-0.035206,0.002542,9.999968e-01,5,-0.029972,-0.040441
bg-0:15,-0.040210,0.004456,9.999822e-01,5,-0.031035,-0.049385
bg-0:20,-0.042080,0.001664,9.999997e-01,5,-0.038654,-0.045506
bg-0:45,-0.044628,0.001173,9.999999e-01,5,-0.042212,-0.047044


In [22]:
# Get the top 10 models based on the leaderboard
leaderboard = predictor.leaderboard(silent=True)
top_10_models = leaderboard['model'].head(10).tolist()

# Prepare the test data without the target column for prediction
test_data_no_target = test_data.drop(columns=[target], errors='ignore')

# Make predictions with each top model and save to individual submission files
for model in top_10_models:
    predictions = predictor.predict(test_data_no_target, model=model)
    submission = pd.DataFrame({'id': test_data['id'], target: predictions})
    submission_file = f"submission_{model}.csv"
    submission.to_csv(submission_file, index=False)
    print(f"Submission file saved for model {model} as {submission_file}.")

Submission file saved for model WeightedEnsemble_L3 as submission_WeightedEnsemble_L3.csv.
Submission file saved for model LightGBMXT_BAG_L2 as submission_LightGBMXT_BAG_L2.csv.
Submission file saved for model LightGBM_BAG_L2 as submission_LightGBM_BAG_L2.csv.
Submission file saved for model CatBoost_BAG_L2 as submission_CatBoost_BAG_L2.csv.
Submission file saved for model XGBoost_BAG_L2 as submission_XGBoost_BAG_L2.csv.
Submission file saved for model RandomForestMSE_BAG_L2 as submission_RandomForestMSE_BAG_L2.csv.
Submission file saved for model ExtraTreesMSE_BAG_L2 as submission_ExtraTreesMSE_BAG_L2.csv.
Submission file saved for model WeightedEnsemble_L2 as submission_WeightedEnsemble_L2.csv.
Submission file saved for model LightGBM_BAG_L1 as submission_LightGBM_BAG_L1.csv.
Submission file saved for model LightGBMXT_BAG_L1 as submission_LightGBMXT_BAG_L1.csv.
